## 1. Importing Libraries

In [ ]:
# All imports here
import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
from datetime import datetime
import copy

## 2. Paths to Dataset

In [ ]:
# Define the paths here
one_million_path = "/content/drive/MyDrive/Sudoku_Solver/data/1Million.csv"
nine_million_path = "/content/drive/MyDrive/Sudoku_Solver/data/9Million.csv"

## 3. Reading the Dataset

In [ ]:
# Read in pandas
nine_million_dataset = pd.read_csv(nine_million_path,  nrows=2000000)

In [ ]:
nine_million_dataset.columns

Index(['puzzle', 'solution'], dtype='object')

## 4. Train-Test Data Split

In [ ]:
def train_test_split_data(df, test_ratio=0.2): 
  questions = []
  solutions = []

  # Process questions and solutions
  for idx, row in df.iterrows():
    # Pre process question
    question = row['puzzle']
    question = (np.array(list(map(int,list(question)))).reshape((9,9,1))/9) - 0.5
    # print(question.shape)
    questions.append(question)

    # Pre process solutions
    solution = row['solution']
    solution = (np.array(list(map(int,list(solution)))).reshape(81, 1)) - 1
    # print("Solution", solution.shape)
    solutions.append(solution)

  x_train, x_test, y_train, y_test = train_test_split(np.array(questions), np.array(solutions), test_size=test_ratio, random_state=42)

  del(questions)
  del(solutions)
  
  return x_train, x_test, y_train, y_test

In [ ]:
X_train, x_test, Y_train, y_test = train_test_split_data(nine_million_dataset)

In [ ]:
print(len(X_train), len(Y_train), len(x_test), len(y_test))

1600000 1600000 400000 400000


In [ ]:
X_train.shape

(1600000, 9, 9, 1)

## 5. Building the Model

In [ ]:
def build_model():

  model = Sequential()

  # Layer 1
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
  model.add(BatchNormalization())

  # Layer 2
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
  model.add(BatchNormalization())

  # Layer 3
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
  model.add(BatchNormalization())

  # Layer 4
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
  model.add(BatchNormalization())

  # Layer 5
  model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

  model.add(Flatten())
  model.add(Dense(81*9))
  model.add(Reshape((-1, 9)))
  model.add(Activation('softmax'))

  adam = Adam(learning_rate=.001)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

  return model

In [ ]:
sudoku_cnn = build_model()
sudoku_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 9, 9, 64)          640       
                                                                 
 batch_normalization (BatchN  (None, 9, 9, 64)         256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          36928     
                                                                 
 batch_normalization_1 (Batc  (None, 9, 9, 64)         256       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 64)          36928     
                                                                 
 batch_normalization_2 (Batc  (None, 9, 9, 64)         2

##6. Training the Model

In [ ]:
sudoku_cnn.fit(X_train, Y_train, batch_size=128, epochs=15)

Epoch 1/15
50000/50000 [==============================] - 288s 6ms/step - loss: 0.4317 - accuracy: 0.7985
Epoch 2/15
50000/50000 [==============================] - 279s 6ms/step - loss: 0.3777 - accuracy: 0.8210
Epoch 3/15
50000/50000 [==============================] - 279s 6ms/step - loss: 0.3700 - accuracy: 0.8261
Epoch 4/15
50000/50000 [==============================] - 277s 6ms/step - loss: 0.3657 - accuracy: 0.8288
Epoch 5/15
50000/50000 [==============================] - 277s 6ms/step - loss: 0.3630 - accuracy: 0.8306
Epoch 6/15
50000/50000 [==============================] - 277s 6ms/step - loss: 0.3610 - accuracy: 0.8318
Epoch 7/15
50000/50000 [==============================] - 277s 6ms/step - loss: 0.3595 - accuracy: 0.8327
Epoch 8/15
50000/50000 [==============================] - 278s 6ms/step - loss: 0.3584 - accuracy: 0.8334
Epoch 9/15
50000/50000 [==============================] - 278s 6ms/step - loss: 0.3575 - accuracy: 0.8340
Epoch 10/15
50000/50000 [=====================

## 7. Solving Sudoku cell by cell using CNN

In [ ]:
class SudokuCNN:

  def get_accuracy(self,feats, labels, model): 
    start = datetime.now()
    correct = 0
    for i,feat in enumerate(feats): 
      pred = self.inference_sudoku(feat, model)
      true = labels[i].reshape((9,9))+1
      if(abs(true - pred).sum()==0):
          correct += 1         
    print(correct/feats.shape[0])
    print(datetime.now()-start) 

  def denormalize(self,a):
      return (a+.5)*9

  def normalize(self,a):
      return (a/9)-.5

  def inference_sudoku(self,sample, model):
    feat = copy.copy(sample)
    while(1):
      out = model.predict(feat.reshape((1,9,9,1)))  
      out = out.squeeze()

      pred = np.argmax(out, axis=1).reshape((9,9))+1 
      prob = np.around(np.max(out, axis=1).reshape((9,9)), 2) 
      
      feat = self.denormalize(feat).reshape((9,9))
      mask = (feat==0)
    
      if(mask.sum()==0):
          break
          
      prob_new = prob*mask

      ind = np.argmax(prob_new)
      x, y = (ind//9), (ind%9)

      val = pred[x][y]
      feat[x][y] = val
      feat = self.normalize(feat)
    
    return pred

##9. Train accuracy

In [ ]:
sudoku_predictor = SudokuCNN()
sudoku_predictor.get_accuracy(X_train[:100], Y_train[:100], sudoku_cnn)

##10. Test Accuracy

In [ ]:
sudoku_predictor.get_accuracy(x_test[:100], y_test[:100], sudoku_cnn)

0.92
0:03:31.891644


In [ ]:
sudoku_cnn.save("/content/drive/MyDrive/Sudoku_Solver/data/model_{2M}_{15E}_{92A}.h5")

## 11. Predicting the results for 100 Sudoku problems

In [ ]:
def predict_blanks(x_test, y_test):

  start = datetime.now()
  correct = 0
  error = 0

  for question, solution in zip(x_test, y_test):

    out = solver1.predict(question.reshape(1, 9, 9, 1))
    out = out.squeeze()

    predicted = np.argmax(out, axis=1) + 1
    question = ((question+0.5)*9).reshape(81, -1)
    indices = np.where(question == 0)[0]

    solution = solution.squeeze()
    # print([indices])

    diff = abs(predicted[indices]-solution[indices])
    error += len(np.where(diff != 0)[0])/len(indices)
    if abs(solution-predicted).sum() == 0:
      correct += 1

  print("Total Error: ", error)
  print("Percentage", error/100)
  print("Time Taken: ", datetime.now()-start)
  print("Correct Solutions: ", correct)

Total Error:  96.33389342214711
Percentage 0.9633389342214711
Time Taken:  0:00:03.666041
Correct Solutions:  0


In [ ]:
predict_blanks(x_test[:100],y_test[:100])

##11. Backtracking Algorithm to Solve Sudoku

In [ ]:
class Backtrack:

    def solve_sudoku(self, board) -> None:
        if self.backtrack(board, 0, 0):
          return

    def denormalize(self,a):
      return (a+.5)*9
                    
    def backtrack(self, board, r: int, c: int) -> bool:
        # Go to next empty space
        while board[r][c] != 0:
            c += 1
            if c == 9: c, r = 0, r+1
            if r == 9: return True
        # Try all options, backtracking if not work
        for k in range(1, 10):
            if self.is_valid_sudoku_move(board, r, c, k):
                board[r][c] = k
                if self.backtrack(board, r, c):
                    return True
        board[r][c] = 0
        return False
    
    def is_valid_sudoku_move(self, board, r: int, c: int, cand: int) -> bool:
        # Check row
        if any(board[r][j] == cand for j in range(9)): return False
        # Check col
        if any(board[i][c] == cand for i in range(9)): return False
        # Check block
        br, bc = 3*(r//3), 3*(c//3)
        if any(board[i][j] == cand for i in range(br, br+3) for j in range(bc, bc+3)): return False
        return True

##12. Evaluating Backtrack Algorithm

In [ ]:
def evaluate_backtrack(x_test):

  backtrack = Backtrack()
  backtrack_test = backtrack.denormalize(x_test)
  start = datetime.now()

  print(backtrack_test.shape)

  for puzzle in backtrack_test:
    backtrack.solve_sudoku(puzzle.reshape(9,9))

  print("Time taken: ", datetime.now() - start)

(100, 9, 9, 1)
Time taken:  0:00:04.665841


In [ ]:
evaluate_backtrack(x_test[:100])